# Pipeline for Full-Frame Images
1. Convert the cutout into a lightcurve
2. Make a river plot of the lightcurve and draw a line at the extrema
3. Produce a Lomb-Scargle periodogram and label the harmonics

## Setup and Imports
This cell imports all of the necessary libraries. It also sets up two important variables: `DOWNLOAD_PATH` and `LIGHTCURVE_PATH`, where the cutouts are downloaded and the lightcurves are stored. On Colab, these need to be changed.

In [2]:
import os
import json
import random
import tess_cpm #custom functions
import numpy as np
import pandas as pd
import lightkurve as lk

import matplotlib
import matplotlib.pyplot as plt

from astropy import table
from astropy import units as u
from scipy.signal import find_peaks
from astroquery.mast import Tesscut
from astroquery.mast.utils import parse_input_location

from collections import OrderedDict

%matplotlib inline

DOWNLOAD_PATH = "./data"
LIGHTCURVE_PATH = "./lightcurves"
PROCVER = "v0.3.1"

## Downloading and Processing Images

This cell contains the code to download a TESS cutout and process it with `tess-cpm`. The resulting lightcurve is saved to a CSV. The name of the folder in which the lightcurve is saved is its TIC, and the filename is the sector number. 

**`download_object(coordinates=None, size=50, sector=None, inflate=True, objectname=None)`**

Downloads a cutout for a given sector and returns the path. If the cutout has already been downloaded, only returns the path.

**`process_ffi_lc(tic, sector)`**

Generates a lightcurve from a full-frame image of the object with the given TIC during the given sector. Saves the lightcurve to a CSV and returns it.

**`load_ffi_lc(tic, sector)`**

Loads the lightcurve of the given object and sector. Downloads and processes the FFI for the sector if it does not already exist.

**`load_lcs(tic, sectors)`**

Loads several sectors for the given tic into one combined lightcurve. `sectors` should be a list of integers.


In [4]:
def download_object(coordinates=None, size=50, sector=None, inflate=True, objectname=None):
    print(f"Downloading {objectname}:{sector}")
    coords = parse_input_location(coordinates, objectname)
    ra = f"{coords.ra.value:.6f}"

    path = Tesscut.download_cutouts(coordinates=coordinates, size=size, sector=sector, path=DOWNLOAD_PATH, inflate=inflate, objectname=objectname)
    return [path[0]["Local Path"]]

def get_cycle(sector):
    cycles = [(1, 13), (14, 26), (27, 39), (40, 55), (56, 69), (70, 83), (83, 96)]
    for i, cycle in enumerate(cycles):
        if sector in range(cycle[0], cycle[1] + 1):
            return i + 1

def process_ffi_lc(tic, sector):
    """
    Create a lightcurve for a given TIC and sector. Save it as a CSV.
    """

    # If the lightcurve already exists, do nothing.
    if os.path.isfile(f"{LIGHTCURVE_PATH}/{tic.split()[0]}_{tic.split()[1]}/Sector{sector}.csv"):
        return

    # Download the full-frame images from TESScut and run TESS-CPM.
    file = download_object(objectname=tic, sector=sector)
    s = tess_cpm.Source(f"{file[0]}", remove_bad=True)
    
    s.set_aperture(rowlims=[25, 25], collims=[25, 25])
    s.add_cpm_model(exclusion_size=5, n=64, predictor_method="similar_brightness")

    s.set_regs([0.1])
    s.holdout_fit_predict(k=100)

    apt_detrended_flux = s.get_aperture_lc(data_type="cpm_subtracted_flux")

    # Create the LightCurve object from the time and flux returned by TESS-CPM
    lc = lk.LightCurve(time=s.time, flux=apt_detrended_flux)

    # Fill the LightCurve object's metadata dictionary.
    lc.meta["SECTOR"] = sector
    lc.meta["TESSID"] = tic.split()[1]
    lc.meta["TARGETID"] = tic.split()[1]
    lc.meta["LABEL"] = tic
    lc.meta["OBJECT"] = tic
    lc.meta["PROCVER"] = PROCVER
    lc.meta["AUTHOR"] = "TESS"
    lc.meta["CREATOR"] = "JMAG-CPM"
    lc.meta["CYCLE"] = get_cycle(sector)

    # Make a directory to store the lightcurve
    if not os.path.isdir(f"{LIGHTCURVE_PATH}/{tic.split()[0]}_{tic.split()[1]}/"):
        os.mkdir(f"{LIGHTCURVE_PATH}/{tic.split()[0]}_{tic.split()[1]}/")

    # Save the metadata to lightcurves/[TIC]/Sector[NUMBER].meta
    with open(f"{LIGHTCURVE_PATH}/{tic.split()[0]}_{tic.split()[1]}/Sector{sector}.meta", "w") as f:
        f.write(json.dumps(lc.meta))

    # Save the lightcurve to lightcurves/[TIC]/Sector[NUMBER].csv
    lc.to_csv(f"{LIGHTCURVE_PATH}/{tic.split()[0]}_{tic.split()[1]}/Sector{sector}.csv")
    
    return lc # return the lightcurve

def load_ffi_lc(tic, sector):
    """Load a lightcurve for a given TIC and sector"""
    path = f"{LIGHTCURVE_PATH}/{tic.split()[0]}_{tic.split()[1]}/Sector{sector}.csv" # Path of the lightcurve's csv file
    if not os.path.isfile(path): # If the lightcurve doesn't already exist, create it using process_ffi_lc
        process_ffi_lc(tic, sector)

    # Load the lightcurve and metadata into a LightCurve object and return it.
    csv = pd.read_csv(path) 
    lc = lk.LightCurve(time=csv.time, flux=csv.flux) 
    with open(f"{LIGHTCURVE_PATH}/{tic.split()[0]}_{tic.split()[1]}/Sector{sector}.meta") as f:
        lc.meta = json.loads(f.read(), object_pairs_hook=OrderedDict)
    return lc

def load_lcs(tic, sectors):
    lcs = []
    for sector in sectors: 
        lcs.append(load_lc(tic, sector))

    return lk.LightCurveCollection(lcs).stitch(corrector_func=lambda x:x)

## Downloading Premade LCs
works with either TIC or EPIC ID
bottom two functions here are not working yet due to issue with fits file unit read in

examples: 
- get_lc("TIC", 59129133, 43, "SPOC", 120)
- get_lc("EPIC", 246676629, 13, "K2", 1800)

In [6]:
def download_premade_lc(id_type, starid, secorcamp, author, exptime): #made by sarah
    name=id_type+" "+str(int(starid)) #id_type must be TIC or EPIC and starid is just a number
    if id_type=="TIC":
        search_result = lk.search_lightcurve(name,author=author,exptime=exptime,sector=secorcamp)
    elif id_type=="EPIC":
        search_result = lk.search_lightcurve(name,author=author,exptime=exptime,campaign=secorcamp)
    lc=search_result.download()
    return lc

def process_premade_lc(id_type, starid, secorcamp, author, exptime): #made by sarah
    secorcampname=''
    if id_type=='TIC':
        secorcampname='Sector'
    elif id_type=='EPIC':
        secorcampname='Campaign'
    # If the lightcurve already exists, do nothing and exit function.
    if os.path.isfile(f"{LIGHTCURVE_PATH}/{id_type}_{starid}/{secorcampname}{secorcamp}.fits"):
        return

    #download lc
    lc=download_premade_lc(id_type, starid, secorcamp, author, exptime)

    # Make a directory to store the lightcurve
    if not os.path.isdir(f"{LIGHTCURVE_PATH}/{id_type}_{starid}/"):
        os.mkdir(f"{LIGHTCURVE_PATH}/{id_type}_{starid}/")

    # Save the lightcurve as a fits file
    lc.to_fits(f"{LIGHTCURVE_PATH}/{id_type}_{starid}/{secorcampname}{secorcamp}.fits")

    return lc

def load_premade_lc(id_type, starid, secorcamp, author, exptime): #made by sarah
    secorcampname=''
    if id_type=='TIC':
        secorcampname='Sector'
    elif id_type=='EPIC':
        secorcampname='Campaign'
    path = f"{LIGHTCURVE_PATH}/{id_type}_{starid}/{secorcampname}{secorcamp}.fits" # Path of the lightcurve's csv file
    if not os.path.isfile(path): # If the lightcurve doesn't already exist, create it using process_sector
        process_premade_lc(id_type, starid, secorcamp, author, exptime)
        
    # Load the lightcurve into a LightCurve object and return it.
    lc=lk.read(path)
    fluxunit=lc.columns['flux'].unit
    print(fluxunit)
    if fluxunit=='electron / s':
        print('bad unit')
    lc.columns['flux'].unit=u.electron / u.s
    return lc

## Loading Either Type of LC
designed to take strings from Best LCs table like (all for same object):
- TIC-59129133-43-SPOC-120
- TIC-59129133-5-FFI-30min
- EPIC-246676629-13-EVEREST-1800

In [8]:
def get_lc_string(infostring): #made by sarah
    parts=infostring.split("-")
    id_type=parts[0]
    starid=parts[1]
    secorcamp=parts[2]
    author=parts[3]
    exptime=parts[4]
    name=id_type+" "+str(int(starid)) #id_type must be TIC or EPIC and starid is just a number
    if author=='FFI':
        lc=load_ffi_lc(name,secorcamp)
    else:
        lc=download_premade_lc(id_type, starid, secorcamp, author, exptime)
    return lc

## Graphing

These functions produce stylized graphs based on the images Mark showed us. By default, they will fold the lightcurve unless `fold` is set to `False`.

In [10]:
def graph_lc(lc, ylim=None, epoch_time=None, period=None, title=None, ax=None, normalize=True):
    fig = None
    if period == None:
        period = lc.to_periodogram(maximum_period=5).period_at_max_power
    if epoch_time == None:
        epoch_time = lc.time[0]
    if normalize:
        lc=lc.normalize()
    
    lc = lc.fold(period, epoch_time)
    lc = lc.remove_outliers(sigma=3)
    
    blc = lc.bin(u.Quantity(25, u.s))
    
    plt.style.use("seaborn-v0_8-notebook")

    if ax == None:
        fig, ax = plt.subplots()

    ax.scatter(lc["time"].value, lc["flux"], 4, "#1f77b4", alpha=0.1)
    ax.scatter(blc["time"].value, blc["flux"], 6, "#1f77b4", alpha=1)
    
    ax.set_title(title if title is not None else lc.meta["LABEL"])
    ax.set_xlabel("Phase")
    if normalize:
        ax.set_ylabel("Normalized Flux")
    else:
        ax.set_ylabel("Flux")

    if ylim is not None:
        ax.set_ylim(-ylim, ylim)
    
    if fig != None:
        return fig

In [11]:
def graph_lc_paper(lc, ylim=None, epoch_time=None, period=None, ax=None, normalize=True):
    fig = None
    if period == None:
        period = lc.to_periodogram().period_at_max_power
    if epoch_time == None:
        epoch_time = lc.time[0]
    if normalize:
        lc=lc.normalize()
        
    lc = lc.fold(period, epoch_time)
    lc = lc.remove_outliers(sigma=3)
    
    blc = lc.bin(u.Quantity(40, u.s))
    
    plt.style.use("seaborn-v0_8-paper")
    if ax == None:
        fig, ax = plt.subplots()
    
    ax.scatter(lc["time"].value, lc["flux"], 14, "#d3d3d3", alpha=0.7)
    ax.scatter(blc["time"].value, blc["flux"], 20, "#1f77b4", alpha=1)
    
    ax.set_title(lc.meta["LABEL"])
    ax.set_xlabel("Phase")
    if normalize:
        ax.set_ylabel("Normalized Flux")
    else:
        ax.set_ylabel("Flux")

    if ylim is not None:
        ax.set_ylim(-ylim, ylim)
    
    if fig != None:
        return fig

## Graphing Lightcurves for Multiplot

In [13]:
def subgraph_phaselc(lc, ylim=None, epoch_time=None, period=None, ax=None, bintime=25, showperiod=True): #made by sarah
    fig = None
    if period == None:
        period = lc.to_periodogram(maximum_period=5).period_at_max_power
    if epoch_time == None:
        epoch_time = lc.time[0]
    # if normalize:
    #     lc=lc.normalize()
    
    lc = lc.fold(period, epoch_time)
    lc = lc.remove_outliers(sigma=3)
    
    blc = lc.bin(u.Quantity(bintime, u.s)) #default bins to every 25 seconds but can change that
    
    plt.style.use("seaborn-v0_8-paper")

    if ax == None:
        fig, ax = plt.subplots(figsize=(5,4))

    ax.scatter(lc["time"].value, lc["flux"], 4, "#000000", alpha=0.05)
    ax.scatter(blc["time"].value, blc["flux"], 6, "#000000", alpha=1)
    ax.margins(0) #getting rid of unnecessary whitespace

    if showperiod==True:
        ax.text(0.95, 0.05, s="Period: "+str(round(period.value,4)*period.unit), transform=ax.transAxes, ha='right')
    
    ax.set_xlabel("Phase")
    #if normalize:
    ax.set_ylabel("Normalized Flux")
    # else:
    #     ax.set_ylabel("Flux")

    if ylim is not None:
        ax.set_ylim(-ylim, ylim)
    
    if fig != None:
        return fig

def subgraph_2phaselc(lc, ylim=None, epoch_time=None, period=None, ax=None, bintime=25): #made by sarah
    fig = None
    if period == None:
        period = lc.to_periodogram(maximum_period=5).period_at_max_power
    if epoch_time == None:
        epoch_time = lc.time[0]
    # if normalize:
    #     lc=lc.normalize()
    
    lc = lc.fold(period, epoch_time)
    lc = lc.remove_outliers(sigma=3)
    maxmin=lc.phase.max()-lc.phase.min()
    lc1=lc[lc.phase<0] #left half
    lc2=lc[lc.phase>0] #right half
    lc1.time=lc1.time+maxmin #shift right by full period
    lc2.time=lc2.time-maxmin #shift left
    
    blc = lc.bin(u.Quantity(bintime, u.s)) #default bins to every 25 seconds but can change that
    blc1=blc[blc.time<0]
    blc2=blc[blc.time>0]
    blc1.time=blc1.time+maxmin
    blc2.time=blc2.time-maxmin
    #blc1 = lc1.bin(u.Quantity(bintime, u.s))
    #blc2 = lc2.bin(u.Quantity(bintime, u.s))
    
    plt.style.use("seaborn-v0_8-paper")

    if ax == None:
        fig, ax = plt.subplots(figsize=(5,2))

    ax.scatter(lc["time"].value, lc["flux"], 4, "#000000", alpha=0.05)
    ax.scatter(lc1["time"].value, lc1["flux"], 4, "#000000", alpha=0.05)
    ax.scatter(lc2["time"].value, lc2["flux"], 4, "#000000", alpha=0.05)
    ax.scatter(blc["time"].value, blc["flux"], 6, "#000000", alpha=1)
    ax.scatter(blc1["time"].value, blc1["flux"], 6, "#000000", alpha=1)
    ax.scatter(blc2["time"].value, blc2["flux"], 6, "#000000", alpha=1)
    ax.margins(0) #getting rid of unnecessary whitespace
    
    ax.set_xlabel("Phase with Period x2")
    #if normalize:
    ax.set_ylabel("Normalized Flux")
    # else:
    #     ax.set_ylabel("Flux")

    if ylim is not None:
        ax.set_ylim(-ylim, ylim)
    
    if fig != None:
        return fig

def subgraph_hphaselc(lc, ylim=None, epoch_time=None, period=None, ax=None, bintime=25): #made by sarah
    #h for half phase
    fig = None
    if period == None:
        period = lc.to_periodogram(maximum_period=5).period_at_max_power
    if epoch_time == None:
        epoch_time = lc.time[0]
    # if normalize:
    #     lc=lc.normalize()
    
    lc = lc.fold(period, epoch_time)
    lc = lc.remove_outliers(sigma=3)
    lc1=lc[lc.phase<0]
    lc2=lc[lc.phase>0]
    minphase=lc1.phase.min()
    lc2.time=lc2.time+minphase
    
    blc1 = lc1.bin(u.Quantity(bintime, u.s)) #default bins to every 25 seconds but can change that
    blc2 = lc2.bin(u.Quantity(bintime, u.s))
    
    plt.style.use("seaborn-v0_8-paper")

    if ax == None:
        fig, ax = plt.subplots(figsize=(5,2))

    ax.scatter(lc1["time"].value, lc1["flux"], 4, "#000000", alpha=0.05)
    ax.scatter(lc2["time"].value, lc2["flux"], 4, "#000000", alpha=0.05)
    ax.scatter(blc1["time"].value, blc1["flux"], 6, "#000000", alpha=1)
    ax.scatter(blc2["time"].value, blc2["flux"], 6, "#000000", alpha=1)
    ax.margins(0) #getting rid of unnecessary whitespace
    
    ax.set_xlabel("Phase with Periodx0.5")
    #if normalize:
    ax.set_ylabel("Normalized Flux")
    # else:
    #     ax.set_ylabel("Flux")

    if ylim is not None:
        ax.set_ylim(-ylim, ylim)
    
    if fig != None:
        return fig

def subgraph_fulllc(lc, ylim=None, ax=None): #made by sarah
    fig = None
    # if normalize:
    #     lc=lc.normalize()

    plt.style.use("seaborn-v0_8-paper")

    if ax == None:
        fig, ax = plt.subplots(figsize=(10,2))
    ax.scatter(lc["time"].value, lc["flux"], 1, "#000000", alpha=1)
    ax.margins(0)

    ax.set_xlabel("Time [BJD]")
    #if normalize:
    ax.set_ylabel("Normalized Flux")
    # else:
    #     ax.set_ylabel("Flux")

    if ylim is not None:
        ax.set_ylim(-ylim, ylim)

    if fig != None:
        return fig

## Text for multiplot

In [15]:
def subgraph_text(lc, infostring, gaiatable, id_table, ax=None): #made by sarah
    #gaiatable and id_table must be a dataframe. pass through multiplot fn.
    fig = None
    
    parts=infostring.split("-")
    id_type=parts[0]
    starid=float(parts[1])
    secorcamp=parts[2]
    author=parts[3]
    exptime=parts[4]
    
    if ax == None:
        fig, ax = plt.subplots()

    if id_type=='TIC':
        secorcampname='Sector'
        gaiarow=gaiatable[gaiatable.TIC==starid]
        idrow=id_table[id_table.TIC==starid]
    elif id_type=='EPIC':
        secorcampname='Campaign'
        gaiarow=gaiatable[gaiatable.epic_id==starid]
        idrow=id_table[id_table.epic_id==starid]

    ninety=np.percentile(lc['flux'],90)
    ten=np.percentile(lc['flux'],10)
    amplitude=ninety-ten

    if np.isnan(idrow.TIC.values[0]):
        TIC='none'
    else:
        TIC=int(idrow.TIC.values[0])

    if np.isnan(idrow.epic_id.values[0]):
        epic='none'
    else:
        epic=int(idrow.epic_id.values[0])

    if np.isnan(idrow.age_Myr.values[0]):
        age='unknown'
    else:
        age=f'{int(idrow.age_Myr.values[0])} Myr'

    try:
        textstring=f'''{secorcampname} {secorcamp}
LC Author: {author}
Cadence: {exptime}
    
TIC: {TIC}
EPIC: {epic}
GDR3: {int(gaiarow.gaiadr3_source_id.values[0])}

Discovery Paper: {gaiarow.author.values[0]} {int(gaiarow.year.values[0])}

Age: {age}
Stellar Group: {gaiarow.group.values[0]}

RA: {round(gaiarow.ra.values[0],2)}, DEC: {round(gaiarow.dec.values[0],2)} degrees
G: {round(gaiarow.g_absmag.values[0],2)}, RP: {round(gaiarow.rp_absmag.values[0],2)}, 
    BP: {round(gaiarow.bp_absmag.values[0],2)} abs mag
G-RP: {round(gaiarow.abs_g_rp.values[0],2)}
RUWE: {round(gaiarow.ruwe.values[0],2)}
Parallax: {round(gaiarow.parallax.values[0],2)} mas
Distance: {round(gaiarow.distance.values[0],2)} pc
Flux Amplitude (90th-10th): 
    {amplitude:.4f}'''

    except: #version for when there's no gaia data
        textstring=f'''{secorcampname} {secorcamp}
LC Author: {author}
Cadence: {exptime}
    
TIC: {TIC}
EPIC: {epic}
GDR3: no gaia data

Discovery Paper: {idrow.author.values[0]} {int(idrow.year.values[0])}

Age: {age}
Stellar Group: {idrow.group.values[0]}

Flux Amplitude (90th-10th): 
    {amplitude:.4f}'''    

    ax.axis('off') #hide axes
    ax.text(0, 1, textstring, va='top', ha='left', transform=ax.transAxes, fontsize=9)

    if fig != None:
        return fig

## Complex Rotator Check

Compares periodogram peak power values with estimated harmonic values to verify complex rotator identity.

In [17]:
def harmonic_checker(lc,max_period=5,percentage=0.15):
    from astropy.table import QTable, Table, Column
    from scipy.signal import find_peaks
  #retrieve the period of the lightcurve

    lightcurve_pg = lc.to_periodogram(maximum_period=max_period)
    
    period_lc = lightcurve_pg.period_at_max_power.value
   
  #manually find the harmonics of our lightcurve  
    harmonics_lightcurve_pg = []
    for i in range(1,9): #consider making percentage threshold lower for higher number harmonics 
        harmonics_lightcurve_pg.append(period_lc/i)
        
  #now find the harmonics using the find peaks function
    
    sector_peaks, s_ = find_peaks(lightcurve_pg.power, distance=120,height=lightcurve_pg.max_power.value*percentage)

    sector_peak_periods = []
    for index in sector_peaks:
        sector_peak_periods.append(lightcurve_pg.period[index].value)
  #now we compare both harmonics within 10% and store our data into a dictionary
    Keys = ['Harmonic','Period','Power','Relative Power']
    my_dicts = []
    Normalized_Power = s_['peak_heights']/lightcurve_pg.max_power.value
    
    for y,test_period in enumerate(sector_peak_periods):
        for i in range(len(harmonics_lightcurve_pg)):
            
            response = False
  #loop for comparison   
            if 0.9*harmonics_lightcurve_pg[i] <= test_period <= 1.10*harmonics_lightcurve_pg[i]:
                response =  True
                my_dict={}
  #loop that identifies and stores the main harmonic  
            if i == 0 and response == True:
                my_dict['Harmonic']= 'Main Harmonic'
                my_dict['Period']=(test_period)
                my_dict['Power']=s_['peak_heights'][y]
                my_dict['Relative Power']= Normalized_Power[y]
    
                my_dicts.append(my_dict)
  #loop that stores the following harmonics into a dictionary                
            elif response == True:
                my_dict['Harmonic']= 'Harmonic '+str(i)
                my_dict['Period']=(test_period)
                my_dict['Power']=s_['peak_heights'][y]
                my_dict['Relative Power']= Normalized_Power[y]
    
                my_dicts.append(my_dict)
            
  #asking the function to create a table and a "result" of the type of lightcurve we have from our dictionary's data  
    my_dicts
    lightcurve_Table = Table(rows=my_dicts)
    Number_of_spikes = len(lightcurve_Table)
    if Number_of_spikes ==1:
        return "Not Complex",lightcurve_Table
    elif Number_of_spikes ==2:
        return "Double Dip",lightcurve_Table
    elif Number_of_spikes > 2:
        return "Complex",lightcurve_Table

In [18]:
def is_complex(lc):
    complex, _ = harmonic_checker(lc)
    return (complex == "Complex")

## Graph Lombscargle

Creates a Lombscargle periodogram that is annotated with evidence that implies the existence of a complex rotator.

In [20]:
def lombscargle(lc, period=None, ax=None):
    fig = None
    if period == None:
        period = lc.to_periodogram(maximum_period=5).period_at_max_power
        period=period.value
    pg = lc.to_periodogram(maximum_period=1.2*period)
    harmonics_lightcurve_pg = []
    for i in range(1,5):
        harmonics_lightcurve_pg.append(period/i)
    
    peaks, _ = find_peaks(pg.power, distance = 120, height = pg.max_power.value*0.15)
    y = pg.power[peaks] #defines y-values as the powers corresponding to the indexes in peaks
    x = pg.period[peaks] #defines x-values as the periods corresponding to the indexes in peaks
    
    if ax == None:
        fig, ax = plt.subplots()
    pg.plot(ax=ax, c="black")
    ax.scatter(x, y, c="red", marker="x")
    for i, period in enumerate(harmonics_lightcurve_pg):
        color = "green" if i == 0 else "red"
        ax.axvline(period, alpha=0.2, linewidth=9-(1.4*i), color=color, zorder=0)

    if fig != None:
        return fig

def subgraph_pg(lc, period=None, ax=None): #made by sarah
    fig = None
    # if normalize:
    #     lc=lc.normalize()
    if period == None:
        period = lc.to_periodogram(maximum_period=5).period_at_max_power
    period=period.value
    pg = lc.to_periodogram(maximum_period=1.2*period)
    harmonics_lightcurve_pg = []
    for i in range(1,5):
        harmonics_lightcurve_pg.append(period/i)

    plt.style.use("seaborn-v0_8-paper")
    
    peaks, _ = find_peaks(pg.power, distance = 120, height = pg.max_power.value*0.15)
    y = pg.power[peaks] #defines y-values as the powers corresponding to the indexes in peaks
    x = pg.period[peaks] #defines x-values as the periods corresponding to the indexes in peaks
    
    if ax == None:
        fig, ax = plt.subplots(figsize=(5,4))
    pg.plot(ax=ax, c="black")
    ax.get_legend().remove()
    ax.scatter(x, y, c="red", marker="x")
    ax.margins(0)
    for i, period in enumerate(harmonics_lightcurve_pg):
        color = "blue" if i == 0 else "red"
        ax.axvline(period, alpha=0.2, linewidth=9-(1.4*i), color=color, zorder=0)

    if fig != None:
        return fig

## River Plot

Generates an annotated river plot of the given lightcurve and returns the matplotlib axes.

In [22]:
def river_plot(lc, epoch_time=None, period=None, ax=None):
    fig = None
    
    if period == None:
        period = lc.to_periodogram(maximum_period=5).period_at_max_power
    if epoch_time == None:
        epoch_time = lc.time[0]

    lc = lc.fold(period, epoch_time=epoch_time)    
    blc = lc.bin(u.Quantity(period/200, u.d))
    blc.sort("flux")

    min = blc[0][0].value/lc.period.value
    max = blc[-1][0].value/lc.period.value

    if ax == None:
        fig, ax = plt.subplots()
    
    lc.plot_river(ax=ax)
    
    ax.axvline(min, 0, 1, color="red")
    ax.axvline(max, 0, 1, color="red")
    
    if fig != None:
        return fig

In [23]:
def subgraph_rp( #modified by sarah from source code plot_river function
        lc,
        period=None,
        epoch_time=None,
        ax=None,
        bin_points=1,
        minimum_phase=-0.5,
        maximum_phase=0.5,
        method="mean",
        #normalize=True,
        **kwargs,
    ) -> matplotlib.axes.Axes:

        fig = None
    
        if hasattr(lc, "time_original"):  # folded light curve
            time = lc.time_original
        else:
            time = lc.time

        if period == None:
            period = lc.to_periodogram(maximum_period=5).period_at_max_power
        if epoch_time == None:
            epoch_time = lc.time[0]
        # if normalize:
        #     lc=lc.normalize()

        foldedlc = lc.fold(period, epoch_time=epoch_time) 
        blc = foldedlc.bin(u.Quantity(period/200, u.d))
        blc.sort("flux")
        min = blc[0][0].value/foldedlc.period.value
        max = blc[-1][0].value/foldedlc.period.value

        if (bin_points == 1) and (method in ["mean", "median"]):
            bin_func = lambda y, e: (y[0], e[0])
        elif method == "mean":
            bin_func = lambda y, e: (np.nanmean(y), np.nansum(e**2) ** 0.5 / len(e))

        s = np.argsort(time.value)
        x, y, e = time.value[s], lc.flux[s], lc.flux_err[s]
        med = np.nanmedian(lc.flux)
        e /= med
        y /= med

        # Here `ph` is the phase of each time point x
        # cyc is the number of cycles that have occurred at each time point x
        # since the phase 0 before x[0]
        n = int(
            period.value
            / np.nanmedian(np.diff(x))
            * (maximum_phase - minimum_phase)
            / bin_points
        )
        if n == 1:
            bin_points = int(maximum_phase - minimum_phase) / (
                2 / int(period.value / np.nanmedian(np.diff(x)))
            )
            warnings.warn(
                "`bin_points` is too high to plot a phase curve, resetting to {}".format(
                    bin_points
                ),
                LightkurveWarning,
            )
            n = 2
        ph = x / period.value % 1
        cyc = np.asarray((x - x % period.value) / period.value, int)
        cyc -= np.min(cyc)

        phase = (epoch_time.value % period.value) / period.value
        ph = ((x - (phase * period.value)) / period.value) % 1
        cyc = np.asarray(
            (x - ((x - phase * period.value) % period.value)) / period.value, int
        )
        cyc -= np.min(cyc)
        ph[ph > 0.5] -= 1

        ar = np.empty((n, np.max(cyc) + 1))
        ar[:] = np.nan
        bs = np.linspace(minimum_phase, maximum_phase, n + 1)
        cycs = np.arange(0, np.max(cyc) + 2)

        ph_masks = [(ph > bs[jdx]) & (ph <= bs[jdx + 1]) for jdx in range(n)]
        qual_mask = np.isfinite(y)
        for cyc1 in np.unique(cyc):
            cyc_mask = cyc == cyc1
            if not np.any(cyc_mask):
                continue
            for jdx, ph_mask in enumerate(ph_masks):
                if not np.any(cyc_mask & ph_mask & qual_mask):
                    ar[jdx, cyc1] = np.nan
                else:
                    ar[jdx, cyc1] = bin_func(
                        y[cyc_mask & ph_mask], e[cyc_mask & ph_mask]
                    )[0]

    
        # If the method is average we need to denormalize the plot
        if method in ["mean", "median"]:
            median = np.nanmedian(lc.flux.value)
            if hasattr(median, "mask"):
                median = median.filled(np.nan)
            ar *= median

        d = np.max(
            [
                np.abs(np.nanmedian(ar) - np.nanpercentile(ar, 5)),
                np.abs(np.nanmedian(ar) - np.nanpercentile(ar, 95)),
            ]
        )
        vmin = kwargs.pop("vmin", np.nanmedian(ar) - d)
        vmax = kwargs.pop("vmax", np.nanmedian(ar) + d)
        if method in ["mean", "median"]:
            cmap = kwargs.pop("cmap", "viridis")

        with plt.style.context("seaborn-v0_8-paper"):
            if ax == None:
                fig, ax = plt.subplots(figsize=(12, cyc.max() * 0.1))
            # if ax is None:
            #     _, ax = plt.subplots(figsize=(12, cyc.max() * 0.1))

            im = ax.pcolormesh(
                bs, cycs, ar.T, vmin=vmin, vmax=vmax, cmap=cmap, **kwargs
            )
            #documentation https://matplotlib.org/stable/gallery/axes_grid1/demo_colorbar_with_axes_divider.html
            divider = make_axes_locatable(ax)
            cax = divider.append_axes("right", size="3%", pad="1%")
            if fig is not None: #so ax was none before and not part of a multiplot
                cbar = fig.colorbar(im, cax=cax)
            else:
                cbar = ax.figure.colorbar(im, cax=cax)

            ax.axvline(min, 0, 1, color="red")
            ax.axvline(max, 0, 1, color="red")
            
            if method in ["mean", "median"]:
                unit = "[Normalized Flux]"
                if lc.flux.unit is not None:
                    if lc.flux.unit != u.dimensionless_unscaled:
                        unit = "[{}]".format(lc.flux.unit.to_string("latex"))
                if bin_points == 1:
                    cbar.set_label("Normalized Flux")
                else:
                    cbar.set_label("Average Flux in Bin {}".format(unit))

            ax.set_xlabel("Phase")
            ax.set_ylabel("Cycle")
            ax.set_ylim(cyc.max(), 0)
            p2=round(period.value/2,2)
            p4=round(period.value/4,2)
            ax.set_xticks(ticks=[-0.5,-0.25,0,0.25,0.5],labels=[-p2,-p4,0,p4,p2])
            a = cyc.max() * 0.1 / 12.0
            b = (cyc.max() - cyc.min()) / (bs.max() - bs.min())
            #ax.set_aspect(a / b)
            
        if fig != None:
            return fig

## Mini CMD

In [25]:
def subgraph_CMD(infostring,gaiatable,ax=None): #made by sarah
    #gaiatable must be a dataframe
    fig = None
    
    parts=infostring.split("-")
    id_type=parts[0]
    starid=float(parts[1])
    secorcamp=parts[2]
    author=parts[3]
    exptime=parts[4]
    
    if ax == None:
        fig, ax = plt.subplots()
        
    ax.scatter(gaiatable.abs_g_rp,gaiatable.g_absmag,marker='.',c='darkgray',alpha=0.05)
    if id_type=='TIC':
        ax.scatter(gaiatable[gaiatable.TIC==starid].abs_g_rp,gaiatable[gaiatable.TIC==starid].g_absmag,marker='X',s=75,c='deeppink')
    elif id_type=='EPIC':
        ax.scatter(gaiatable[gaiatable.epic_id==starid].abs_g_rp,gaiatable[gaiatable.epic_id==starid].g_absmag,marker='X',s=75,c='deeppink')
    ax.invert_yaxis()
    ax.set_xlim(0.7,1.9)
    ax.set_ylim(18,3)
    ax.set_ylabel(r'M$_G$ [mag]')
    ax.set_xlabel(r'$G-G_{\text{RP}}$ [mag]')

    if fig != None:
        return fig

## Multiplots

In [27]:
def multiplot(lc):
    fig, axs = plt.subplot_mosaic([
        ['tl', 'tr'],
        ['b', 'b']
    ], figsize=(16, 12))
    
    river_plot(lc, ax = axs["tl"])
    axs["tl"].set_title("")
    
    lombscargle(lc, ax = axs["tr"])
    axs["tr"].get_legend().remove()
    
    graph_lc_paper(lc, ax = axs["b"])

    return fig

In [28]:
def multiplot2(lc, infostring, gaiatable, period=None, bintime=None): #made by sarah
    fig, axs = plt.subplot_mosaic([
        ['A1', 'A2'],
        ['A1', 'B2'],
        ['C1', 'C2'],
        ['D1', 'D2'],
        ['E1', 'E1']
    ], figsize=(16, 16), layout="constrained")

    if period==None:
        period = lc.to_periodogram(maximum_period=5).period_at_max_power
    if bintime==None:
        bintime=u.Quantity(period/200,u.s)

    lc=lc.normalize() #normalize once now instead of having to do it in every method
    
    subgraph_phaselc(lc, ax=axs['A1'], period=period, bintime=bintime)
    
    subgraph_pg(lc, ax = axs['C1'], period=period)

    subgraph_2phaselc(lc, ax=axs['A2'], period=period, bintime=bintime)
    subgraph_hphaselc(lc, ax=axs['B2'], period=period, bintime=bintime)

    subgraph_rp(lc,ax=axs['C2'], period=period)

    subgraph_CMD(infostring,gaiatable,ax=axs['D1'])

    subgraph_text(lc, infostring,gaiatable,ax=axs['D2'])
    
    subgraph_fulllc(lc, ax = axs['E1'])

    return fig

In [29]:
def multiplot3(lc, infostring, gaiatable, id_table, period=None, bintime=None): #made by sarah
    # ['11','12','13','14'],
    # ['21','22','23','24'],
    # ['31','32','33','34'],
    # ['41','42','43','44'],
    # ['51','52','53','54'],
    # ['61','62','63','64'],
    # ['71','72','73','74'],
    # ['81','82','83','84'],
    # ['91','92','93','94'],
    # ['101','102','103','104']
    
    fig, axs = plt.subplot_mosaic([
        ['11','11','11','14'],
        ['11','11','11','14'],
        ['11','11','11','14'],
        ['11','11','11','14'],
        ['51','51','53','53'],
        ['61','61','53','53'],
        ['71','71','73','73'],
        ['71','71','73','73'],
        ['91','91','91','91'],
        ['91','91','91','91']
    ], figsize=(10,16), layout="constrained")

    if period!=None:
        period=u.Quantity(period,u.d) #if period is passed, assign it a unit
    else:
        period = lc.to_periodogram(maximum_period=5).period_at_max_power
    if bintime==None:
        bintime=u.Quantity(period/200,u.s)

    parts=infostring.split("-")
    id_type=parts[0]
    starid=float(parts[1])
    secorcamp=parts[2]
    author=parts[3]
    exptime=parts[4]

    if author=='FFI':
        lc.flux=lc.flux+1 #shift normalization to be around 1
    else:
        lc=lc.normalize() #normalize once now instead of having to do it in every method
    
    subgraph_phaselc(lc, ax=axs['11'], period=period, bintime=bintime)
    
    subgraph_pg(lc, ax = axs['73'], period=period)

    subgraph_2phaselc(lc, ax=axs['51'], period=period, bintime=bintime)
    subgraph_hphaselc(lc, ax=axs['61'], period=period, bintime=bintime)

    subgraph_rp(lc,ax=axs['53'], period=period)

    subgraph_CMD(infostring,gaiatable,ax=axs['71'])

    subgraph_text(lc, infostring,gaiatable, id_table=id_table, ax=axs['14'])
    
    subgraph_fulllc(lc, ax = axs['91'])

    return fig

## Other Functions

In [31]:
def complex_rotator_check(tic,show=True):
    from astropy.io import ascii
 
    results = []
    available_sectors = (get_sectors(tic))['sectors'] #retrieving avaliable sectors list from our tic
    list_of_sectors = np.array(available_sectors)
    print('Analyzing '+str(len(list_of_sectors[list_of_sectors > 56]))+' sectors:') #letting us know how many sectors we are analyzing
    for i in range(len(available_sectors)):      
        if available_sectors[i] > 56: #loop that searches through our list and picks out all avaliable sectors after 56
            print('Analyzing sector '+str(available_sectors[i])+'for '+tic)#letting us know what's happening
            lc = load_lc(tic = tic, sector=available_sectors[i]) #loads that tic and sector's lc
            multiplot(lc) #plots the data from that lc
            plt.savefig(f"Panels/{tic}_sector{available_sectors[i]}.png",dpi=150)
            if show == False:
                plt.close()
            
            result, table = harmonic_checker(lc) #plugs our lc into harmonic checker to get a result and table
            print('This sector is: '+str(result)) #writing the result for us to see
            table_data=table #turning table into a format for astrp
            ascii.write(table_data,f"Harmonic_Tables/{tic}_sector{available_sectors[i]}_harmonics.csv",format='csv',fast_writer=False,overwrite=True)
            results.append((result, available_sectors[i]))
    result_data=results
    ascii.write(result_data,f"Complex_Results/{tic}_complex_results.csv",format='csv',fast_writer=False,overwrite=True)
    return result_data
    

In [32]:
def get_sectors(tic):
    """Returns a list of available sectors given an TIC."""
    sectors = []
    for result in lk.search_tesscut(str(tic)):
        sectors.append(int(result.mission[0].split(" ")[2]))
        
    return {"tic": tic, "sectors": sectors}

def get_targets(tics):
    """Constructs a dictionary of TICs and available sectors from a list of TICs"""
    targets = []

    for tic in tics:
        targets.append(get_sectors(tic))

    return targets